In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import glob
import datetime as dt
import multiprocessing as mp
from datetime import datetime
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas import Series
from tqdm import tqdm, tqdm_notebook
import time
import sys
import matplotlib.pyplot as plt

!pip install missingno
import missingno as msno

In [3]:
cd gdrive/My Drive/TFM/

/content/gdrive/My Drive/TFM


#method definintion

In [4]:
def returns(candles_close_prices):
    return pd.Series(np.diff(np.log(candles_close_prices)))

#Data loading

In [ ]:
# raw trade data from https://public.bitmex.com/?prefix=data/trade/ 
Dollar_bars = pd.DataFrame()
for i,file in enumerate(glob.glob("data/bars/new_features/*.csv")):
  if i == 0:
    Dollar_bars = Dollar_bars.append(pd.read_csv(file))
    print('Percentge of files already Loaded:',round((i/len(glob.glob("data/bars/new_features/*.csv")))*100,1), '%. There are', len(glob.glob("data/bars/new_features/*.csv"))-i, "files left", end='')
  else:
    Dollar_bars = Dollar_bars.append(pd.read_csv(file))
    print('\r Percentge of files already Loaded:',round((i/len(glob.glob("data/bars/new_features/*.csv")))*100,1), '%. There are', len(glob.glob("data/bars/new_features/*.csv"))-i, "files left",end='', flush=True)

 Percentge of files already Loaded: 98.9 %. There are 1 files left

In [ ]:
Dollar_bars.drop(columns=['Unnamed: 0','timestamp.1'],inplace=True)
# timestamp parsing
Dollar_bars['timestamp'] = Dollar_bars.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%d %H:%M:%S.%f"))

Dollar_bars.index = Dollar_bars['timestamp']
Dollar_bars


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#Missing values and outlier study

In [ ]:
aux = returns(Dollar_bars['close'])

Dollar_bars['returns'] = np.append([np.nan],aux)

Dollar_bars[['mean', 'std']] = Dollar_bars['returns'].ewm(span=31) \
                               .agg(['mean', 'std'])
Dollar_bars.dropna(inplace=True)
                             
msno.matrix(Dollar_bars)


,open,high,low,close,size,return,mean,std,outlier,returns
timestamp,,,,,,,,,,
2021-06-01 01:20:45.697399,2701.90,2705.95,2655.00,2655.00,72674,-0.017437,-0.013652,0.005531,0,NaN
2021-06-01 02:07:18.093862,2654.95,2659.25,2615.55,2645.70,74175,-0.003509,-0.010050,0.007084,0,-0.003509
2021-06-01 02:36:01.273498,2645.70,2664.00,2631.05,2664.00,73945,0.006893,-0.005396,0.010429,0,0.006893
2021-06-01 03:15:11.830451,2664.00,2677.85,2614.80,2638.45,74897,-0.009637,-0.006357,0.009101,0,-0.009637
2021-06-01 03:55:54.484884,2638.45,2659.10,2617.85,2654.85,74948,0.006197,-0.003913,0.009684,0,0.006197
...,...,...,...,...,...,...,...,...,...,...
2021-08-27 21:38:22.068676,3280.95,3285.00,3272.00,3278.25,45161,-0.000823,0.000604,0.002880,0,-0.000823
2021-08-27 21:53:02.366899,3278.95,3280.15,3256.05,3265.85,45249,-0.003790,0.000329,0.002990,0,-0.003790
2021-08-27 22:27:46.999295,3265.90,3282.00,3264.05,3278.45,45298,0.003851,0.000549,0.003022,0,0.003851


In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(go.Scatter(
    x=Dollar_bars.index,
    y=Dollar_bars['return'],
    name="return",
    mode = 'lines+text',
    textfont_family="Arial_Black"),
    secondary_y=True,

)
fig.add_trace(go.Scatter(
    x=Dollar_bars.index,
    y=Dollar_bars['mean'],
    name="mean",
    mode = 'lines+text',
    textfont_family="Arial_Black"),
    secondary_y=True,

)
fig.add_trace(go.Scatter(
    x=Dollar_bars.index,
    y=2*Dollar_bars[ 'std'],
    name="2.5 Standar deviations",
    mode = 'lines+text',
    textfont_family="Arial_Black"),
    secondary_y=True,

)
fig.add_trace(go.Scatter(
    x=Dollar_bars.index,
    y=-2*Dollar_bars[ 'std'],
    name="-2.5 Standar deviations",
    mode = 'lines+text',
    textfont_family="Arial_Black"),
    secondary_y=True,

)

In [ ]:
cond = (Dollar_bars['returns'] > Dollar_bars['mean'] + Dollar_bars['std'] * 2) | (Dollar_bars['returns'] < Dollar_bars['mean'] - Dollar_bars['std'])
Dollar_bars['outliers'] = np.where(cond, 1, 0)

In [ ]:
outliers.head()

In [ ]:
Dollar_bars['outlier'].value_counts()[0]

4104

In [ ]:
Dollar_bars['outlier'].value_counts() 
print(round(Dollar_bars['outlier'].value_counts()[1]/(Dollar_bars['outlier'].value_counts()[0]+Dollar_bars['outlier'].value_counts()[1])*100,
            3),
      '% of the samples detected as outliers')

2.749 % of the samples detected as outliers


In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(go.Scatter(
    x=Dollar_bars.index,
    y=Dollar_bars['return'],
    name="return",
    mode = 'lines+text',
    textfont_family="Arial_Black"),
    secondary_y=True,

)
fig.add_trace(go.Scatter(
    x=outliers.index,
    y=outliers['returns'],
    name="returns",
    mode = 'markers',
    textfont_family="Arial_Black"),
    secondary_y=True,

)